In [1]:
import features

Importamos todas las librerias necesarias para el projecto, todos los modulos de python en el proyecto. Y creamos una instancia de la clase FeatureBuilder, que se encarga de cargar los datos y hacer las transformaciones necesarias para el proyecto.

In [2]:
feature_builder = features.FeatureBuilder(
    '/Users/miguel.acosta/Downloads/Entidad (3) (2) (3).csv'
)

Las transformaciones iniciales consisten en eliminar las columnas que no se van a utilizar, y en renombrar las columnas que se van a utilizar. Tambien calcular la variable dependiente de interes "target" para realizar los modelos


In [3]:
df = feature_builder.read_data()
df = feature_builder.preprocess_column_names(df)
df = feature_builder.create_target(df)

/Users/miguel.acosta/Projects/luisa_coding_task/features.py:77: DtypeWarning: Columns (38,39,52,87) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(self.file_path, sep=";", encoding="latin-1")


Realizamos ingenieria de attributos "feature engineering" sobre los valores para cada columna, para esto se utilizo la siguiente serie de pasos:
1. Convertir cada variable categorica en valores enteros applicando un mapa, este mapa ayuda a identificar la relacion entre el valor de la variable categorica y un valor entero. ejemplo, si la variable categorica es "A", el valor entero es 1, si la variable categorica es "B", el valor entero es 2, y asi sucesivamente.
2. Debido a la informacion dada en la ficha tecnica de la tarea, se applico un mapa ordinal sobre la variable "estrato" ya que esta variable tiene un orden natural, y se asigno un valor entero a cada estrato.
3. Se convirtieron las variables con fechas en respectivos objetos en python que contengan fecha usando el modulo `datetime`.

In [4]:
df = feature_builder.apply_feature_engineering(df)

## Seleccion de variables

Para realizar la seleccion de variables se utiliza el algoritmo de [Recursive Feature Elimination](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)

El algoritmo de eliminación recursiva de características (RFE, por sus siglas en inglés) es un método de selección de características que funciona eliminando gradualmente las características que no son importantes. Se utiliza comúnmente en Machine Learning para mejorar el rendimiento de los modelos y reducir la dimensionalidad de los datos.

RFE iterativamente entrena un modelo, evalúa la importancia de cada característica y elimina la menos importante. Luego, repite el proceso con el conjunto de características restante hasta que se alcance un número deseado de características o se cumpla un criterio de parada.

Para realizar lo anterior se necesitan inputar los datos faltantes, para esto se utilizo el siguiente metodo:
1. Se normalizan las variables de tal manera que esten entre el intervalo 0 y 1
2. Para identificar la "ausencia" de datos, se inputa con -1 en los valores faltantes. Esto permite a los modelos identificar los casos en donde se desconoce o se tiene incertiudumbre sobre el valor de una variable, en vez de asumir de manera arbitraria un valor en particular*

*Ya que el objetivo inicial es encontrar que variables son mas importantes se realiza el procedimiento descrito con anterioridad. Luego en el caso del modelamiento los datos faltantes se inputaran con el algoritmo `KNNImputer`, que se encarga de inputar los valores faltantes basados en los valores de las variables mas cercanas.


In [5]:
# Crear una copia de los datos con -1 imputado en los faltantes
feature_selection_df = features.get_minus_one_imputed_data(df)

# Aplicamos el RFE en los datos con -1 imputado para identificar las 10 variables mas importantes
# esto se hace para reducir el tiempo de computo en el modelamiento

In [6]:
important_features = list(
    features.apply_recursive_feature_elimination(
        feature_selection_df,
        target="BM",
        num_features=10
    ).columns
)
print(important_features)

['fondeador', 'saldo_anterior', 'total_fianza', 'pagaduría', 'cartera_vencida', 'cartera_deteriorada', 'cartera_deteriorada_2', 'altura', 'edad_cliente', 'mora_máxima']
